In [11]:
import networkx as nx
from bokeh.io import show, output_file
from bokeh.models import Plot,ColumnDataSource, Label, LabelSet, Range1d, MultiLine, Circle, TapTool, OpenURL, HoverTool, CustomJS, Slider, Column
from bokeh.models.graphs import from_networkx, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
import copy
from bokeh.plotting import figure
import matplotlib.pyplot as plt
from bokeh.models.glyphs import Circle

In [1]:
import pandas as pd
data = pd.read_excel('Downloads/dp-export-249516.xlsx')
data

Theme Target  Weight
0     A     U1       4
1     B     U2       1
2     A     U3       2
3     B     U4       5
4     C     U1       3
5     C     U5       3

In [29]:
def make_dataset(theme_list):
        by_theme = pd.DataFrame()
        for i, theme_name in enumerate(theme_list):

            subset = data[data['Theme'] == theme_name]
            by_theme = by_theme.append(subset)
            
        return ColumnDataSource((by_theme).reset_index().drop(columns=['index']))

In [32]:
dict(make_dataset(['A']).data)

{'index': array([0, 1], dtype=int64),
 'Theme': array(['A', 'A'], dtype=object),
 'Target': array(['U1', 'U3'], dtype=object),
 'Weight': array([4, 2], dtype=int64)}

{'index': array([0, 1, 2, 3], dtype=int64), 'Theme': array(['A', 'A', 'B', 'B'], dtype=object), 'Target': array(['U1', 'U3', 'U2', 'U4'], dtype=object), 'Weight': array([4, 2, 1, 5], dtype=int64)}
{'index': array([0, 1, 2, 3], dtype=int64), 'Theme': array(['A', 'A', 'B', 'B'], dtype=object), 'Target': array(['U1', 'U3', 'U2', 'U4'], dtype=object), 'Weight': array([4, 2, 1, 5], dtype=int64)}


In [2]:
themes_available = list(data['Theme'].unique())
themes_available

['A', 'B', 'C']

In [6]:
pd.DataFrame(src.data).drop(columns=['index'])

NameError: name 'src' is not defined

In [20]:
import networkx as nx
from bokeh.models.graphs import from_networkx
from bokeh.models import Range1d, MultiLine, Circle, TapTool, Plot, HoverTool, BoxSelectTool
from bokeh.plotting import figure, output_file, save
from bokeh.io import curdoc
from bokeh.layouts import row
from bokeh.models.widgets import Dropdown
from bokeh.events import Tap
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Slider, CheckboxGroup, CustomJS, ColumnDataSource, CDSView
from bokeh.layouts import column, row, WidgetBox
from bokeh.io import curdoc, show, output_notebook
from bokeh.io import output_notebook # enables plot interface in J notebook
# init bokeh
output_notebook()

def make_dataset(theme_list):
        by_theme = pd.DataFrame()
        for i, theme_name in enumerate(theme_list):

            subset = data[data['Theme'] == theme_name]
            by_theme = by_theme.append(subset)
        G = nx.from_pandas_edgelist(by_theme, source='Target', target='Theme')
        graph = from_networkx(
                                G,
                                nx.circular_layout,
                                scale=1,
                                center=(0,0)
                            )
        
        node_colors = []
        for node in G:
            if node in data["Theme"].values:
                node_colors.append("salmon")
            else:
                node_colors.append("deeppink")


        node_size = []
        for node in G:
            if node in data["Theme"].values:
                node_size.append(20)
            else:
                node_size.append(10)
                
#         graph.node_renderer.data_source.data['node_size'] = node_size
#         graph.node_renderer.data_source.data['node_colors'] = node_colors
#         graph.node_renderer.glyph = Circle(size = 'node_size', fill_color = 'node_colors')
                
        d = dict(); 
        d['node'] = graph.node_renderer.data_source.data
        d['edge']   = graph.edge_renderer.data_source.data
        d['node_size']=node_size
        d['node_color']=node_colors
#         print(graph.node_renderer.data_source.data)
#         print(graph.edge_renderer.data_source.data)
        return d
    
def modify_doc(doc):
#     def choose_node_outline_colors(nodes_clicked):
#         outline_colors = []
#         for node in G.nodes():
#             if str(node) in nodes_clicked:
#                 outline_colors.append('pink')
#             else:
#                 outline_colors.append('black')
#         return outline_colors


    def update_node_highlight(attr, old, new):
#         nodes_clicked_ints = source.selected.indices
#         nodes_clicked = list(map(str, nodes_clicked_ints))
#         print(nodes_clicked)
        carriers_to_plot = [checkbox.labels[i] for i in 
                            checkbox.active]
#         print(carriers_to_plot)
        
        updated_data= make_dataset(carriers_to_plot)
        source.data = dict(updated_data['node'])
        graph.edge_renderer.data_source.data = dict(updated_data['edge'])
        graph.node_renderer.data_source.data['node_size'] = (updated_data['node_size'])
        graph.node_renderer.data_source.data['node_colors'] = (updated_data['node_color'])
        
#         print(source.data)


    net_graph = nx.from_pandas_edgelist(data, source='Target', target='Theme')
    
    node_colors = []
    for node in net_graph:
        if node in data["Theme"].values:
            node_colors.append("salmon")
        else:
            node_colors.append("deeppink")


    node_size = []
    for node in net_graph:
        if node in data["Theme"].values:
            node_size.append(20)
        else:
            node_size.append(10)
                
    plot = Plot(plot_width=1000, plot_height=550,
                x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
    graph = from_networkx(
        net_graph,
        nx.circular_layout,
        scale=1,
        center=(0,0)
    )

    # Create nodes and edges
    source = graph.node_renderer.data_source
#     print(source.data)
#     source.data['line_color'] = choose_node_outline_colors('1')
#     graph.node_renderer.glyph = Circle(size=30, line_color="line_color")
#     graph.edge_renderer.glyph = MultiLine(line_alpha=1.6, line_width=0.5)
    
    graph.node_renderer.data_source.data['node_size'] = node_size
    graph.node_renderer.data_source.data['node_colors'] = node_colors
    graph.node_renderer.glyph = Circle(size = 'node_size', fill_color = 'node_colors')#'

#     graph_plot.renderers.append(graph_setup)
    # Add tap tool
    TOOLTIPS = [
        ("Index", "@index"),
    ]
    plot.add_tools(HoverTool(tooltips=TOOLTIPS))

    plot.renderers.append(graph)

#     taptool = plot.select(type=TapTool)

#     plot.on_event(Tap, update_node_highlight)
    
    checkbox = CheckboxGroup(labels=themes_available, active = [0,1,2])
    checkbox.on_change('active', update_node_highlight)
    layout = column(checkbox, plot)
    doc.add_root(layout)
handler=FunctionHandler(modify_doc)
app=Application(handler)

Loading BokehJS ...

In [21]:
show(app)

In [13]:
output_file("output_file_name.html")


In [9]:
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Slider, CheckboxGroup, CustomJS, ColumnDataSource, CDSView
from bokeh.layouts import column, row, WidgetBox


output_notebook()
def modify_doc(doc):
    
    def make_plot(src):
        # Blank plot with correct labels
#     src = make_dataset(['A','B','C'])
        data = pd.DataFrame(src.data).drop(columns=['index'])
        net_graph = nx.from_pandas_edgelist(data, source='Target', target='Theme')
        graph_plot = figure(
                      tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
                    x_range=Range1d(-1.5,1.5), y_range=Range1d(-1.2,1.2), plot_width = 1300,plot_height = 700
            ,background_fill_color='white')
        #graph_plot.circle(source=src)

        for index, row in data.iterrows():
            net_graph.nodes[row['Theme']]['Theme_Mapped'] = row['Theme']
            net_graph.nodes[row['Target']]['Target_Mapped'] = row['Target']

        node_hover_tool = HoverTool(tooltips = [("Value", "@index")]) ##, ("Yearly Count", "@yearly_count")
        graph_plot.add_tools(node_hover_tool)

        node_colors = []
        for node in net_graph:
            if node in data["Theme"].values:
                node_colors.append("salmon")
            else:
                node_colors.append("deeppink")


        node_size = []
        for node in net_graph:
            if node in data["Theme"].values:
                node_size.append(20)
            else:
                node_size.append(10)

        graph_setup = from_networkx(net_graph, nx.spring_layout, scale = 1, center = (0, 0))

        graph_setup.node_renderer.data_source.data['node_size'] = node_size

        graph_setup.node_renderer.data_source.data['node_colors'] = node_colors
        graph_setup.node_renderer.glyph = Circle(size = 'node_size', fill_color = 'node_colors')#'

        graph_plot.renderers.append(graph_setup)
        #graph_plot.legend.click_policy = 'hide'

        # Styling
        #p = style(p)

        return graph_plot
    def update(attr, old, new):
        # Get the list of carriers for the graph
        carriers_to_plot = [carrier_selection.labels[i] for i in 
                            carrier_selection.active]

        # Make a new dataset based on the selected carriers and the 
        # make_dataset function defined earlier
        new_src = make_dataset(carriers_to_plot)
        print(new_src.data)

        # Update the source used the quad glpyhs
        src.data.update(new_src.data)
        print(src.data)
        #p = make_plot(src)
        
    carrier_selection = CheckboxGroup(labels=themes_available, active = [0,2])
    carrier_selection.on_change('active', update)
    print(carrier_selection.active)
    controls = WidgetBox(carrier_selection)
    
    initial_carriers = [carrier_selection.labels[i] for i in carrier_selection.active]
    
    src = make_dataset(initial_carriers)
    
    p = make_plot(src)
    
    layout = column(controls, p)
    doc.add_root(layout)

    
# Set up an application
handler = FunctionHandler(modify_doc)
app = Application(handler)

Loading BokehJS ...